# 5 Throughput

**Note: Throughput productivity calculations and plots for NWR, OR, and SWR are produced for specific loopgroups by the *3b_Corridor_Analysis_loops* module. It is only necessary to use this module for ER throughput data; this module will not longer be necessary if/when ER data is added to TracFlow.**

This script generates the throughput productivity stamp graphs for a given region based on the *[region]_[yr1] [yr2] GNB Lost Productivity.xlsx* file for a given region and combination of analysis years. Despite its label as *5_Throughput*, this script does not depend on any of the previous scripts (except *1_Folder_Setup* if you want to use the prescribed directory structure). It only depends on the existence of the lost productivity Excel file. To run the script, edit the Inputs and then run the Main Code cell.


## Inputs

The inputs for this script are:

base_year, curr_year : the base and current analysis years.

ccr : the current CCR (and name of the base folder)

region : the analysis region

user : name of the analyst running the script.

directory : path to the input file (default is * ./[ccr]/1_Data/[region]/3_Throughput Data/*)

xl\_f : name of the input file (default is *[region]\_[base_year] [curr_year] GNB Lost Productivity.xlsx*)

save_dir : path to where the output plots will be saved (default is *./[ccr]/4_Plots/[region]/5_Throughput*)

In [ ]:
base_year = 2015 
curr_year = 2017
ccr = 'CCR 18'
region = 'ER'

user = 'Regan Hansen'

In [ ]:
# If use_defaults is True, then these will be overwritten in the main code
# if you want to set your own then change use_defaults to False
use_defaults = True 

directory = ''
xl_f = ''
save_path = ''

## Main Code

In [ ]:
import numpy as np # 1.14.0
import pandas as pd # 0.22.0
import os 


# plotting
import matplotlib # '2.1.2'
from matplotlib.ticker import NullLocator
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt

# don't change these unless you want to change the default paths!
if use_defaults:

    directory = './%s/1_Data/%s/3_Throughput Data/'%(ccr, region)

    xl_f = '%s_%s %s GNB Lost Productivity.xlsx'%(region, base_year, curr_year)

    save_path = './%s/4_Plots/%s/5_Throughput'%(ccr, region)

# define function to plot lineplot
def plot_LP(SC_df, fname, title):
    
    font = {'ax' : {
            'family': 'Helvetica Neue LT Std',
            'weight': 'roman',
            'size': '6'},
            
            'title' : {
            'family': 'Helvetica Neue LT Std',
            'weight': 'medium',
            'size': '8'}
            }    
    
    matplotlib.rc('font', **font['ax'])
    w, h = [3.58, 1.1]
    lt, rt, bm, tp = [0.085, 0.97, 0.12, 0.8]    
    fig, ax = plt.subplots(figsize = (w,h))
    fig.subplots_adjust(left=lt, right=rt, bottom=bm, top=tp)
    
    ax.plot(SC_df['base'], color = '0.5', lw = 0.75) # Gray is always base year
    ax.plot(SC_df['curr'], color = '0', lw = 0.75) # Black is always current year


    # set gridlines, ticks, and invisible borders
    ax.grid(axis = 'y', lw = 0.25)
    ax.tick_params(axis = 'x', direction = 'in')
    ax.yaxis.set_tick_params(color = '1', pad=0)
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.set_yticks([1,0.8,0.6,0.4,0.2])
    ax.set_yticklabels(['100%', '80%', '60%', '40%', '20%'], 
                       fontdict = font['ax'])
    
    ax.set_ylim(bottom = 0, top = 1)
    ax.set_xticks([5*12, 8*12, 11*12, 14*12, 17*12, 20*12])
    ax.set_xticklabels(['5 AM', '8 AM', '11 AM', '2 PM', '5 PM', '8 PM'], 
                       fontdict = font['ax'])
    ax.set_xlim(left = 5*12, right = 20*12)
    
    ax.set_title(title, loc='left')

    
    # save figure
    pdffig = PdfPages(fname)
    fig.savefig(pdffig, format = 'pdf')
    
    # write pdf metadata
    metadata = pdffig.infodict()
    metadata['Title'] = '{0} Throughput {1}-{2}'.format(title, base_year, curr_year)
    metadata['Author'] = user + '(Generated by PyMAS)'
    metadata['Subject'] = 'Throughput productivity {0} & {1}'.format(base_year, curr_year)
    
    pdffig.close()
    plt.close()


# Open Excel file
LP_xl = pd.ExcelFile(os.path.join(directory,xl_f))

# Loop through sheets
if region == 'ER': 
    sheets = ['Division', 'Hamilton', 'Freya', 'Custer', 'Fancher',
             'Sprague', 'Broadway', 'Park', 'Argonne']
else:
    sheets = [x for x in LP_xl.sheet_names if 'MP' in x]
    
    
for sheet in sheets:
    
    print('Plotting %s'%(sheet))
    
    # read dataframe
    thru_df = pd.read_excel(LP_xl, sheet_name = sheet, header = 6, 
                            skip_footer = 5, index_col = None)

    if region == 'ER':
        # base is [directions].3 and current is [directions].4
        thru_NBEB = 1 - thru_df[['EB.3', 'EB.4']].dropna()
        thru_NBEB.reset_index(drop = True, inplace = True)
        thru_NBEB.columns = ['base', 'curr']

        thru_SBWB = 1 - thru_df[['WB.3', 'WB.4']].dropna()
        thru_SBWB.reset_index(drop = True, inplace = True)
        thru_SBWB.columns = ['base', 'curr']

    else:
        # base is [directions].3 and current is [directions].4
        thru_NBEB = 1 - thru_df[['NB/EB.3', 'NB/EB.4']].dropna()
        thru_NBEB.reset_index(drop = True, inplace = True)
        thru_NBEB.columns = ['base', 'curr']

        thru_SBWB = 1 - thru_df[['SB/WB.3', 'SB/WB.4']].dropna()
        thru_SBWB.reset_index(drop = True, inplace = True)
        thru_SBWB.columns = ['base', 'curr']


    # Call "plot_LP) function for dataframes
    plot_LP(thru_NBEB, os.path.join(save_path, 
                                    '%s NBEB throughput.pdf'%(sheet)), 
            sheet)
    plot_LP(thru_SBWB, os.path.join(save_path, 
                                    '%s SBWB throughput.pdf'%(sheet)), 
            sheet)
